# Notebook to visualize predictions from regression model

Given a trained regression model, this notebook implements visualizing predictions on the test dataset. It requires a downloaded test dataset, the trained model and you have to specify the path to the trained model

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from plate_optim.regression.regression_model import get_net, get_mean_from_velocity_field
from plate_optim.regression.data import HDF5Dataset, extract_mean_std
from plate_optim.regression.train import evaluate
from codeutils.config import get_config
import torch
import seaborn as sns

plt.rc('font', size=7)
os.chdir("../")

config = get_config('configs/regression/50k15.yaml')
fields = config.dataset_keys
valset = HDF5Dataset(config, config.data_paths_test, normalization=True, test=True)
dataloader = torch.utils.data.DataLoader(valset, batch_size=2, shuffle=False)

out_mean, out_std, field_mean, field_std = extract_mean_std(dataloader.dataset)
out_mean, out_std = out_mean.cuda(), out_std.cuda()
field_mean, field_std = field_mean.cuda(), field_std.cuda()

In the next cell, change the path to where you saved the model weights.

In [ ]:
path = 'path/to/model_weights'
net = get_net(conditional=True, len_conditional=3, scaling_factor=32).cuda()
checkpoint = torch.load(path)
net.load_state_dict(checkpoint['model_state_dict'])


## Making a prediction for a random validation sample

In [ ]:
i = np.random.randint(len(valset))
batch = valset.__getitem__(i, flip_x=False, flip_y=False)
image, velocity_field, vel_mean_sq, _condition, frequencies = (batch[field].unsqueeze(0).to('cuda') for field in fields)
vel_mean_sq = vel_mean_sq.mul(out_std).add(out_mean[frequencies])
frequencies_prediction = torch.linspace(-1.0, 1, 300).cuda().unsqueeze(0)
with torch.no_grad():
    net.eval()
    prediction_field = net(image, _condition, frequencies_prediction)
    prediction = get_mean_from_velocity_field(prediction_field, field_mean, field_std, frequencies_prediction)

prediction = prediction.cpu().numpy()
image = image.cpu().numpy()
_condition = _condition.cpu().numpy()
frequencies = frequencies.cpu().numpy()
frequencies_prediction = frequencies_prediction.cpu().numpy()
vel_mean_sq = vel_mean_sq.cpu().numpy()

def unnormalize(x, mean, std):
    return x * std + mean

condition = unnormalize(_condition, config.mean_conditional_param, config.std_conditional_param)[0]

print(f'condition: {condition}')

fig, ax = plt.subplots(1, 2, figsize=(7, 2))
ax[1].plot(unnormalize(frequencies_prediction[0],150.5, 149.5), prediction[0], label='prediction')
ax[1].plot(unnormalize(frequencies[0], 150.5, 149.5), vel_mean_sq[0], label='ground truth')
ax[1].legend()
ax[0].imshow(image[0, 0][::-1], cmap='gray')
ax[0].plot(condition[1] * 181, (1-condition[2]) * 121, 'x', color='red', markersize=10, markeredgewidth=2.5)
ax[0].axis('off')
sns.despine(offset=5, ax=ax[1])
ax[1].grid(which="major", lw=0.2)
ax[1].set_yticks([-20, 0, 20, 40, 60])
ax[1].set_ylabel('Velocity (dB)')
ax[1].set_xticks([0, 100, 200, 300])
ax[1].set_xlabel("Frequency (Hz)")
fig.suptitle(f'spring bc {condition[0]:.2f}')


# compute loss on test set

In [ ]:
class args_obj: pass
args = args_obj()
args.device = 'cuda'
results = evaluate(args, config, net, dataloader, logger=None, epoch=None)